In [1]:
import pandas as pd
import numpy as np

In [2]:
# df is order csv
df = pd.read_csv("orders_3.5mil.csv")
# cf is customer profile csvcf = pd.read_csv("customer_profiles_recency.csv")

# Read in the names of cuisine types for storage in future dataframes
names = cf.columns.values.tolist()

# Get the count of all the customer orders, sorted by customer id
group_counts = df.groupby('customer_id').count()
arr = cf.to_numpy()
arr = arr[:, 1:]

In [3]:
# Make a new list of customer profiles with any customer with less than 10 orders removed
customer_profile_list = []
i = 0
for index, row in group_counts.iterrows():
    if i >= arr.shape[0]:
        break
    if row[0] > 10:
        customer_profile_list.append(arr[i, :])
    i += 1

In [6]:
# Store the new cutoff into a .csv file
cutoff_df = pd.DataFrame(customer_profile_list, columns=names[1:])
cutoff_df.to_csv('customer_profiles_lowest_cutoff.csv')

print(cutoff_df)

                                customer_id  African   Alcohol    Bakery  \
0      0000267e-c83e-4a48-9776-8163eab97b6a      0.0  0.003465  0.104122   
1      00007de5-00ae-454e-9eb2-c5b100a27659      0.0  0.000000  0.095367   
2      00007e2c-cd9c-43d5-9148-60457215dfd8      0.0  0.000000  0.055443   
3      0000a630-1c0a-45db-8516-d07bda7f22dd      0.0  0.000000  0.034887   
4      000118af-5466-4790-be93-f052f355a773      0.0  0.013151  0.025807   
...                                     ...      ...       ...       ...   
41500  1ee237e6-b354-4e65-9f08-da10a7bd029a      0.0  0.000000  0.142857   
41501  1ee24d2b-5be0-41db-abb3-6c3f540bf8f5      0.0  0.000000  0.222222   
41502  1ee251a8-7838-4d06-93ac-f46563a6958b      0.0  0.000000  0.000000   
41503  1ee2cbb8-f9a5-4454-aa19-c11c4d447f08      0.0  0.000000  0.000000   
41504  1ee2f0e4-8f54-4a00-8fbc-57ad13431b12      0.0  0.166228  0.000000   

       Barbecue      Beef  Breakfast & Brunch  Bubble Tea   Burgers   Butcher  \
0     

In [7]:
# Perform cosine similarity between all the cuisine types, store values in a matrix
cosine_sim = np.zeros((54, 54))
for i in range(54):
    for j in range(54):
        a = cutoff_df.iloc[:, i + 1]
        b = cutoff_df.iloc[:, j + 1]
        cosine_sim[i][j] = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [8]:
# Calculate mean and standard deviation
mean = np.mean(cosine_sim)
std = np.std(cosine_sim)

# Use a sigmoid function to remap values so the average is closer to 0.5
cosine_sim = 1 / (1 + np.exp(-(cosine_sim - mean) / std))

# Store the final matrix as a dataframe
final_df = pd.DataFrame(cosine_sim, names[2:], names[2:])

# Hard coded values to remove
# Values are removed to eliminate any chance of poor recommendation
# Values are only removed in one direction, since the reverse direction can still be recommended
final_df.at['Halal','Alcohol'] = 0
final_df.at['Halal','Pork'] = 0
final_df.at['Kosher','Pork'] = 0

# Store the final dataframe into a .csv file
final_df.to_csv('similarity_matrix_final_official.csv')

In [9]:
# One final check on some statistics of the matrix
minimum = np.min(cosine_sim)
mean = np.mean(cosine_sim)  # New mean
std = np.std(cosine_sim)  # New std

print(minimum)
print(mean)
print(std)

0.0
0.47645324396991867
0.12045220710747154
